# Using Langchain, when a user asks Aina to start the routine, return some trigger to the mobile app so that we can change the screen and show the routine to the user


In [8]:
pip show langchain_openai

Name: langchain-openai
Version: 0.0.5
Summary: An integration package connecting OpenAI and LangChain
Home-page: https://github.com/langchain-ai/langchain
Author: 
Author-email: 
License: MIT
Location: /home/jupyter-seema/.local/lib/python3.10/site-packages
Requires: langchain-core, numpy, openai, tiktoken
Required-by: 
Note: you may need to restart the kernel to use updated packages.


In [6]:
from langchain_openai import OpenAI

In [2]:
import os
import chromadb
from langchain_openai import AzureOpenAI, AzureOpenAIEmbeddings
from langchain.chains import SequentialChain, LLMChain
from langchain.prompts import PromptTemplate
from langchain.memory import ConversationBufferMemory
from langchain_community.vectorstores import Chroma

# Environment configuration
os.environ["OPENAI_API_TYPE"] = "azure"
os.environ["OPENAI_API_VERSION"] = "2023-05-15"
os.environ["OPENAI_API_KEY"] = "549d3edec7cb431dbdcf18a6abdbb1a2"
os.environ["AZURE_OPENAI_ENDPOINT"] = "https://seema.openai.azure.com/"

# Embedding and database setup
embeddings = AzureOpenAIEmbeddings(azure_deployment="seema-embedding")
client = chromadb.HttpClient(host="20.118.32.222", port=8000)
vectordb = Chroma(embedding_function=embeddings, collection_name="Amanda", client=client)
vectordb.add_texts(["I have sensitive skin", "I like L'Oreal products", "Never recommend me sunscreen"])

# Skincare routine prompt template
prompt_template = """
Hi, I am your personal AI assistant from AINA! How may I assist you today?

A skincare routine has the following 7 steps:

1. Cleanser
2. Exfoliator
3. Toner
4. Serum
5. Eye cream
6. Moisturizer
7. Sunscreen

Recommend actual skincare products based on the user's preferences, considering their name, age, gender, and skin tone.

Always output in JSON in this format: {{"step #": "product name"}}

Context: Name: {name}, Age: {age}, Gender: {gender}, Skin Tone: {skin_tone}

Question: What is the best skincare routine for {name}, a {age}-year-old {gender} with {skin_tone} skin?

Please answer just like a human, and ask any additional queries to fine-tune your response for a more personalized answer.
"""

skincare_routine_prompt = PromptTemplate(
    input_variables=['name', 'age', 'gender', 'skin_tone'],
    template=prompt_template
)

skincare_memory = ConversationBufferMemory(input_key='skin_tone', memory_key='routine_history')
llm = OpenAI(temperature=0.8)

# DJ corrected
llm = AzureOpenAI(deployment_name="seema-deployment")

skincare_chain = LLMChain(llm=llm, prompt=skincare_routine_prompt, verbose=True, output_key='skincare_routine', memory=skincare_memory)

# TriggerMobileAppChain definition
class TriggerMobileAppChain:
    def __init__(self, backend_api_endpoint):
        self.backend_api_endpoint = backend_api_endpoint

    def process(self, input_data):
        print(f"Sending skincare routine to mobile app via {self.backend_api_endpoint} with data: {input_data}")
        # Placeholder for actual HTTP POST request logic
        return input_data

# Sequential chain setup
def execute_chains(input_data):
    # Execute the skincare chain
    skincare_result = skincare_chain(input_data)
    
    # Check if a skincare routine was provided in the user request
    if "skincare routine" in input_data.get('user_request', '').lower():
        # Assuming skincare_result contains the desired output directly
        # Trigger the mobile app chain with the skincare result
        trigger_chain = TriggerMobileAppChain(backend_api_endpoint="https://yourapi.com/updateRoutine")
        trigger_chain.process(skincare_result)  # Process method is for demonstration; adapt as needed
        print("Trigger sent: Skincare routine has been sent to the mobile app.")
    else:
        print("No trigger: The user did not request a skincare routine.")
    
    return skincare_result


# Example input handling logic
user_request = "Please provide a skincare routine."
if "skincare routine" in user_request.lower():
    input_details = {
        "name": "Amanda",
        "age": "34",
        "gender": "Female",
        "skin_tone": "dry",
        "user_request": user_request
    }
    result = execute_chains(input_details)
    print(f"Skincare Routine for {input_details['name']}:")
    print(result)
else:
    print("No trigger. Responding based on skin type or other query.")


/home/jupyter-daniel/.local/lib/python3.10/site-packages/langchain_core/_api/deprecation.py:117: LangChainDeprecationWarning: The function `__call__` was deprecated in LangChain 0.1.0 and will be removed in 0.2.0. Use invoke instead.
  warn_deprecated(




> Entering new LLMChain chain...
Prompt after formatting:

Hi, I am your personal AI assistant from AINA! How may I assist you today?

A skincare routine has the following 7 steps:

1. Cleanser
2. Exfoliator
3. Toner
4. Serum
5. Eye cream
6. Moisturizer
7. Sunscreen

Recommend actual skincare products based on the user's preferences, considering their name, age, gender, and skin tone.

Always output in JSON in this format: {"step #": "product name"}

Context: Name: Amanda, Age: 34, Gender: Female, Skin Tone: dry

Question: What is the best skincare routine for Amanda, a 34-year-old Female with dry skin?

Please answer just like a human, and ask any additional queries to fine-tune your response for a more personalized answer.


> Finished chain.
Sending skincare routine to mobile app via https://yourapi.com/updateRoutine with data: {'name': 'Amanda', 'age': '34', 'gender': 'Female', 'skin_tone': 'dry', 'user_request': 'Please provide a skincare routine.', 'routine_history': '', 'skinc